# ナウマンを改象

WindowsでもMacでもLinuxでも、Dockerをインストールすれば、以下に説明する手順で、主にwebブラウザ経由で、naumanniをビルドしたり、実行したり、改造したりできる。

# 使い方

## JupyterLab起動

dockerコマンドが使えるとこで、

```
# docker run -d -v /var/run/docker.sock:/var/run/docker.sock -p 8888:8888 -e GRANT_SUDO=yes --user root --name minimal-lab mnagaku/minimal-lab start-notebook.sh --NotebookApp.token=''
```

を実行するとJupyterLabが8888番ポートでアクセスできる状態で起動する。

## JupyterLab接続

webブラウザから、起動したJupyterLabにアクセスする。例えば、操作しているPC上に起動した場合は、localhostの8888番ポートにアクセスすればよいので、

```
http://localhost:8888/lab
```

でアクセスできる。

## readme.ipynbのUpload

webブラウザで、JupyterLabのHome画面が出たら、左上にある「下線に上向き矢印」ボタンから、この文書「readme.ipynb」をアップロードする。アップロードが完了して、Home画面の一覧に「readme.ipynb」が表示されたら、クリックして「readme.ipynb」での操作に移る。

## JupyterLabコンテナ内からdockerコマンドを使えるようにする

念のため、コンテナ内のモジュールをupdateしてから、dockerコマンドをインストールする。


In [ ]:
%%bash
sudo -i
apt-get update
apt-get upgrade -y
apt-get install -y curl
curl -fsSL https://get.docker.com/builds/Linux/x86_64/docker-17.05.0-ce.tgz | tar -xzC /usr/local/bin --strip=1 docker/docker
docker --version

## naumanniのビルド環境を整える

まずはnode.jsから。
node >= v7.5.0 であることを確認する。


In [ ]:
%%bash
sudo -i
curl -sL https://deb.nodesource.com/setup_7.x | sudo -E bash -
apt-get install -y nodejs
nodejs -v

次にyarn。
yarn >= 0.23.4 であることを確認する。


In [ ]:
%%bash
sudo -i
curl -sS https://dl.yarnpkg.com/debian/pubkey.gpg | apt-key add -
echo "deb https://dl.yarnpkg.com/debian/ stable main" | tee /etc/apt/sources.list.d/yarn.list
apt-get update
apt-get install -y yarn
yarn

## naumanniのソースをクローンする


In [ ]:
%%bash
rm -rf naumanni
git clone --depth 1 https://github.com/naumanni/naumanni.git

## naumanniをビルドする


In [ ]:
%%bash
cd naumanni
yarn
yarn run build

## naumanniのコンテナをビルドする


In [ ]:
%%bash
sudo docker rm -f nora
sudo docker rmi nora/naumanni
cd naumanni
sudo docker build -t nora/naumanni .
sudo docker images

## naumanniを実行する

[http://localhost/](http://localhost/)

でアクセスできる。


In [ ]:
%%bash
sudo docker run -d -p 80:80 --name nora nora/naumanni

## 改象

左に表示されている「Files」から、

naumanni > src > js > utils > html.es6

と辿って、html.es6をダブルクリックすると、html.es6を編集するタブが開く。

_expandMastodonStatus()とゆー、サーバから取ってきた呟きを正規化する処理があるので、

```
function _expandMastodonStatus(content) {
  content = 'おっぱい';
```

と1行足すと、みんなの呟きを強制的に書き換えることができる。

書き換え後、ビルド・コンテナのビルド・実行で、動作を確認できる。
